Preliminary installation
------------------------

The libraries we are going to use today in R are simply **interfaces to powerful stand-alone applications.** We are going to download, for example, some sophisticated map making software, but call it from the comfort of R. This is a common design pattern. Data analysts will often prefer to access new software through R commands. Why? Well, it means that someone has translated the functionality of the other software into R and the data analyst doesn't have to learn something entirely new. It also means that the data analyst can leverage all the other code in R to analyze the data that is produced by this other system. So for a variety of reasons, R becomes a kind of integration platform.

Here, we will need a application called GDAL, the "Geospatial Data Abstraction Library". That's a mouthful. We can use anaconda to install it. In your Terminal window (or at your  Anaconda Prompt) type

> **conda install -c anaconda gdal **

> **conda install clangxx_osx-64**

Once that is installed, you can then install three R packages. These are the basis for all the mapping work we'll be doing. They provide an interface to the GDAL and other applications we just installed. 

In [ ]:
install.packages("rgeos", repos='http://cran.us.r-project.org')
install.packages("rgdal", repos='http://cran.us.r-project.org')
install.packages("maptools", repos='http://cran.us.r-project.org')

Finally, we install two high-level packages that build on the base set of tools and make some nice plots, easily. "ggmap" is a sibling of "ggplot2" as you might expect, and so whatever high-level expressions for graphics we have seen, expect the same from "ggmap". Those will come over the weekend. Saving the best for last :)

And now fire them up! We will also load a couple other packages that should be familiar by now. "dplyr" for commands to work with data frames like filter() and select(), "stringr" for our string tools, and "lubridate" to work with dates.

In [ ]:
library(rgdal)
library(maptools)

In [ ]:
library(dplyr)
library(stringr)
library(lubridate)

Visualizing data - Statistical graphics and maps in R
------------------

We've spent a lot of time with old-school plotting frameworks in R, in particular plot() and boxplot() and hist(). Last time, we saw how to customize these and also provided a few other frameworks that "expose" their functionality a bit more cleanly. We saw different metaphors for expressing graphics. From the grid structure of "lattice" dispays to the more bottom-up construction style of "ggplot2". 

In this notebook, we will start with data from the NYPD on crime statistics in the five boroughs and illustrate another important visualization tool - making maps. There are two data sets available from the Open Data portal that we'll work with.

> [NYPD Complaint Data Historic](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i) includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD) from 2006 to the end of last year (2016). 
<br><br>
[NYPD Complaint Data Current YTD](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-YTD/5uac-w243) includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD) for all complete quarters so far this year (2017). 

These data sets are somewhat amazing for their level of detail. Historically, the NYPD has produced [summaries of their data on a precinct level](http://www.nyc.gov/html/nypd/html/crime_prevention/crime_statistics.shtml). We will grab a link to the data from the Open Data portal and use it along with read.csv() to load the most recent data into R. We should also grab the [data dictionary](https://data.cityofnewyork.us/api/views/5uac-w243/files/fc4f191a-6adf-4268-a1ae-8525a8d7027b?download=true&filename=NYPDIncidentLevelDataFootnotes.pdf) to understand what the different columns mean.

In [ ]:
crime = read.csv("https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv",as.is=TRUE)

In [ ]:
dim(crime)

In [ ]:
options(repr.matrix.max.cols=50)

head(crime)

In [ ]:
sample_n(crime,20)

**1. Read over the data dictionary and look at the variables you have. Create some simple summaries of at least 3 variables to convince yourself you understand what is in the data set. That might suggest a summary() of one or more columns, a hist() for a quantitative variable, or table() for a qualitative one. Tell me what you find.**

In [ ]:
# Your code here



**A first plot**

We can have a look at the crime data spatially using standard plotting techniques. Latitude goes on the y-axis and longitude on the x. Here's a simple plot() of the first 5,000 entries in the data set. The first command in the cell below, you'll recall, sets the size of the plot. 

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

plot(Latitude~Longitude,data=head(crime,10000),pch=".")

And using our standard filter() command from "dpyr", we can focus on a borough or a precinct.

In [ ]:
plot(Latitude~Longitude,data=head(filter(crime,BORO_NM=="BRONX"),10000),pch=".")

Even with just 10,000 out of the 360,000 incidents plotted, we see clear outlines of the boroughs and streets within boroughts. 

**Dropping missing values**

We are now going to focus on the spatial aspects of the crime data for the moment, which means we will  want to include only those rows that have non-missing values for the Latitude and Longitude. 

In [ ]:
# Remove any locations that have missing values
crimenona = filter(crime,!is.na(Longitude),!is.na(Latitude))
dim(crimenona)

In [ ]:
(361740-355886)/361740

We are dropping about 1.6% of the data set this way. It doesn't seem like a lot, but whenever we conduct a reduced analysis, we want to make sure that the data we are dropping is not systematically different than the data we are keeping. Or if it is different, be able to describe the difference. 

To make this comparison, you might make a second data set consisting of just the dropped data points. 

In [ ]:
dropped = filter(crime,is.na(Longitude) | is.na(Latitude))
dim(dropped)

And then we compare "crimenona" and "dropped" along some of the columns. Here we compare borough in case one borough or precinct was consistently having problems reporting addresses...

In [ ]:
table(dropped$BORO_NM)
table(crimenona$BORO_NM)

This gives us the number of incidents in each borough for the two data sets. To make it easier to compare the two, we can divide by the total number of data points in each data set to get the proportion of crimes occurring in each borough...

In [ ]:
table(dropped$BORO_NM)/nrow(dropped)
table(crimenona$BORO_NM)/nrow(crimenona)

... and we don't see big differences. Perhaps some wobble in Manhattan. Next we look at the kinds of offenses themselves. This probably should have been our first variable to check if we read the data dictionary closely. Here are the offenses for the dropped data...

In [ ]:
table(dropped$OFNS_DESC)

... and for the non-misisng data. What do you see? Why?

In [ ]:
table(crimenona$OFNS_DESC)

The data dictionary reports "To further protect victim identities, rape and sex crime offenses are not
geocoded, although the precinct of occurrence is still included for precinct-by-precinct
comparisons." This explains the bulk of the difference. To be complete, we would want to check some of the 30 odd remaining points that don't seem to involve sex crimes, but for now, let's press on.

**The SpatialPointsDataFrame**

We want to examine the crime data spatially. This means we want to easily put the points on a map, perhaps look at their proximity to subway stops or other features in the city, or perhaps count the number of events by Neighborhood Tabulation Areas rather than precincts. To do this, we will transform object "crimenona" from a data frame to a SpatialPointsDataFrame. It's a fancier data structure. We can do everything we could do to a data frame (make subsets, for simple summaries, and so on) but also formally associate each row with a point on the map. This lets us deal with the data spatially. 

First, look at the data set before we make the change...

In [ ]:
class(crimenona)
dim(crimenona)

Then, to make the transformation to a spatial object, we simply have to assign "crimenona" a set of "coordinates", one map coordinate for each row of the table, each incident in the data set. Here we use the **formula notation** we saw with "lattice" -- our coordinates are a function of, or are related to, x and y coordinates, using the tilde "~" to describe the relationship. The x and y coordinates we need in this case 
are taken to be the "Longitude" and "Latitude" columns from the "crimenona" data set.

In [ ]:
coordinates(crimenona) = ~Longitude+Latitude
class(crimenona)

In [ ]:
dim(crimenona)

Notice that "crimenona" now something different. It's a new kind of data frame, one that works with SpatialPoints. Notice also that the number of columns has changed. The "Longitude" and "Latitude" are now pulled out into a set of **attributes**. Any object in R can have attributes -- they represent extra information that helps provide context for the data. For a spatial object, attributes include coordinates (one pair of x-y per incident, or row, of the crime data), the "projection" or coordinate system they are referencing, and even a bounding box for the points.

In [ ]:
names(attributes(crimenona))

The Geographic Coordinate System, which uses latitude and longitude to describe locations on the
Earth’s surface, is the coordinate system we're most familiar with. Latitude can be thought of as the lines that intersect the y-axis, and longitude as lines that
intersect the x-axis.  By default, GPS receivers record data in latitude-longitude using the WGS84 (World Geodetic System 1984) datum. If we suppose the earth is a spheroid, then a datum tells us the position of the spheroid relative to the center of the earth. There are lots of datums and when you pick up spatial information from the web -- it's important to know what coordinate system is used as well as the datum it's referenced to.

Here we explicitly assign the coordinate reference system (CRS) to our latitude and longitude points. This will help us later when we have to move from this coordinate system to others. The CRS is just a string that tells R what the numbers in our coordinates actually mean.

In [ ]:
proj4string(crimenona) = CRS("+proj=longlat +datum=WGS84")

Once we have a spatial points data frame, we can make plots of the points more easily, simply passing the name of the object. R knows how to extract x and y. Note also that the aspect ratio of the plot is improved. R can handle that for a spatial object also.

One small detail. For the moment, "dplyr" funtions do not work on Spatial versions of data frames. So for subsetting or adding columns we are, for the moment, back to the primitive way of doing things with dollar signs and square brackets. I'll point out where we are falling back on old techniques. For the moment, just keep in mind that filter() and mutate() and your other favorites won't work on spatial data frames.

In [ ]:
plot(head(crimenona,10000),pch=".")

**Shape files**

Let's bring in some other information. The city publishes various data sets to help you interpret data like our crime information. For example, you can [find descriptions of NYC's political and administrative districts here](http://www1.nyc.gov/site/planning/data-maps/open-data/districts-download-metadata.page). We will choose a simple outline of the city's boroughs. 

For the most part, the city's data are stored in ["shape files"](https://en.wikipedia.org/wiki/Shapefile). A shape file is actually a folder of several files that describes points or lines or regions (polygons) on a map. Download this shape file to the folder where you first started your Jupyter Notebook and unzip it.

> [http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nybb_17d.zip](http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nybb_17d.zip)

You'll see several files in the nybb_16d folder -- a "database file", nybb.dbf, that has information about each borough, and a file describing the boundaries of each borough, nybb.shp.

We read the files into R using a function called readOGR() from "rgdal".

In [ ]:
boros = readOGR("nybb_17d/nybb.shp","nybb")
class(boros)

Because boroughs are regions and not points, we have a SpatialPolygonsDataFrame. It consists of just five rows (five boroughs) and 4 columns. We can have a nice printout using as.data.frame() so "boros" looks like our other data frames.

In [ ]:
dim(boros)

In [ ]:
as.data.frame(boros)

So one row per borough and then attributes that describe the area of each borough. Let's have a look at these shapes.

In [ ]:
plot(boros)

The coordinates() for a collection of points is just the x- and y-locations for each point. When you call coordinates() on a collection of polygons, you get the centers of the polygons. So here are the x- and y-coordinates of the centers for each borough. What do you notice?

In [ ]:
coordinates(boros)

The numbers are enormous. Whereas latitude and longitude are usually in degrees, these numbers are several orders of magnitude larger than what we'd expect. 

The city typically publishes its spatial information using the State Plane Coordinate System. The U.S. State Plane Coordinate System is actually a series of
separate systems, each covering a state, or a part of a state, and is only used in the United States. It
is popular with some state and local governments due to its high accuracy, achieved through the use
of relatively small zones. **The coordinate system here is measured in feet from a fixed location for each region.** Here's a map. Our region is number 3104. Find it!


<img src=http://gis.tulaliptribes-nsn.gov/Portals/8/images/spc_zones_04_lrg.jpg>

We can convert between coordinate systems, assuming we're sure about the details of the measurements. In this case, the particular coordinate system is given in the .prj file of the shape file. Have a look! Use [this conversion web site](http://www.earthpoint.us/StatePlane.aspx) and enter the two numbers (the x and y) from the first entry in the table above for teh coordinates() of "boros". Convert the State Plane points to lat-lon. 

We can also use the function spTransform() to take "boros" from this coordinate system to regular latitude and longitude. Try it out and look at the coordinates(). Make sure that the first one matches what you got from the conversion web site.

In [ ]:
boros = spTransform(boros, CRS("+proj=longlat +datum=WGS84"))
coordinates(boros)

To show that they match up, we can now overlay our incident data on the boro map.

In [ ]:
plot(boros)
points(head(crimenona,10000),pch=".")

To pull everything into one place, here are the important steps from the code above. 

In [ ]:
# Read in the data and transform the coordinate system
boros = readOGR("nybb_17d/nybb.shp","nybb")
boros = spTransform(boros, CRS("+proj=longlat +datum=WGS84"))

# Make a plot with an overlay of the crime data
plot(boros)
points(head(crimenona,10000),pch=".")

**2. Download the shape file describing Neighborhood Tabulation Areas**
>[http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nynta_17d.zip](http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nynta_17d.zip)

**and read it into an R SpatialPolygonsDataFrame called "nbhds". These data also use the State Plane Coordinate System, so you'll have to transform them to the usual lat-lon projection as we did with "boros". You can [read about the data here](http://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page). Then, plot the neighborhoods and add the first 10,000 incidents from "crimenona" over the top.**

In [ ]:
# Put your code here



As a first test, the following command should give you a printout that looks like a standard data frame. You should see columns like the Tabulation Area code and name, as well as the name of the borough.

In [ ]:
head(as.data.frame(nbhds))

As a second test, this command should generate a plot of the city with the Neighborhood Tabulation Areas marked out. Oh and "Morningside Heights" will be in green. We use the ifelse() command to choose between "blue" for a region or "lightgrey". 

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)

plot(nbhds,col=ifelse(nbhds$NTAName == "Morningside Heights","blue","lightgrey"))
text(coordinates(nbhds),labels=nbhds$NTAName,cex=0.5)

**Subsetting, aggregating and joining spatial data**

Unfortunately, at the moment, the "dplyr" functions that work on data frames, don't work on their spatial equivalents. So subsetting has to be done the old fashioned way, using  square brackets and indices or logical expressions to select rows and columns. Here, for example, we keep only the neighborhood that is named "Morningside Heights". This will return a SpatialPolygonsDataFrame that has one row (and one shape).

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)

mh = nbhds[nbhds$NTAName=="Morningside Heights",]
plot(mh)

Again, subsets of an ordinary data frame can be defined using square brackets with a comma to separate row selections from column selections. For spatial data frames, we have a few more exotic options. For example, when you subset the rows of a SpatialPointsDataFrame with a polygon, you keep only those points that fall within the polygon. So here is how we would extract all the crimes happening in Morningside Heights.

In [ ]:
mhcrime = crimenona[mh,]
dim(mhcrime)

In [ ]:
sort(table(mhcrime$OFNS_DESC))

In [ ]:
plot(mh)
points(mhcrime,pch=".")

Next, we would like to "join" the information in our "nbhds" dataset with that in the crime incidents -- it might be helpful  to know which neighborhood the crimes occurred in. We can do this using the command over() (for overlay). This returns a data frame that consists of the attributes from "nbhds" when a point falls into one of the neighborhood areas. If it doesn't, then it returns NA, the missing value. It will give back a data frame that is the same size as the original, in this case "crimenona". (The overlay() command will take a little processing time. It is comparing hundreds of thousands of points.)

In [ ]:
overlay = over(crimenona,nbhds)
head(overlay,20)

In [ ]:
class(overlay)
dim(overlay)

Since "overlay" is data frame, we can use "dplyr" functions. So, for example, we can use group_by() and summarise() on "overlay" to count the number of incidents happening in each neighborhood.

In [ ]:
totals = summarise(group_by(overlay,NTACode),n=n())
head(totals)

**Graphics with ggplot and ggmap**

The maptools library helps us with some basic maps (points, lines, shapes). But there are various graphics systems in R and some of them are good at maps as well. Last time we finished with ggplot (the Grammar of Graphics) and it's flexible way of specifying visualizations. We can use a special "geom" that represents a map to make simple things like choropleth maps.

Below we load up ggplot2...

In [ ]:
library(ggplot2)

Now, ggplot2 doesn't work well with the spatial objects and so you have to change them back to data frames. This can be done with the ggplot2 function fortify(). It is annoying that you have to do this, but this is what happens when different research groups develop different pieces of software. Sometimes they don't go together as seamlessly as you'd like. The structure of one group's data may not fit with the logic of another group.

In [ ]:
b = fortify(nbhds,region="NTACode")

And now we make a ggplot(). Here we create a choropleth map where the colors depend on the total number of crimes reported in the neighborhood area. 

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)

ggplot() + 
   geom_map(aes(map_id = NTACode, fill = n), map = b, data = totals) +
   expand_limits(x = b$long, y = b$lat)+ coord_map()

Similar in spirit to ggplot is ggmap. It uses the same syntax and the same "geoms". The only difference is that you can have an explicit map layer that sits behind the data. Here we plot out the crimes in Morningside Heights.

In [ ]:
install.packages("ggmap", repos='http://cran.us.r-project.org')

In [ ]:
library(ggmap)

In [ ]:
qmap(location="Columbia University",zoom = 14)

You can select different tile mills if you prefer a different look to the map. Here is one from Stamen Design.

In [ ]:
qmap(location="Columbia University",zoom = 14,source="stamen",maptype="toner")

Finally, we get_map() of our neighborhood and then, as we called ggplot() we call ggmap() and add some points. Again, because the gg-family is not friendly with spatial objects, you have to turn the mhcrime SpatialPointsDataFrame into a regular data frame (and irritatingly, fortify() won't do it). I know, janky free software.

In [ ]:
morningside = get_map(location="Columbia University",zoom = 14)

ggmap(morningside)+geom_point(aes(x=Longitude,y=Latitude),data=as.data.frame(mhcrime))

**Last comment on joining data**

Above we computed the crime report totals for each neighborhood area. We can now merge() the data back with the "nbhds" spatial information. merge() is an old-school R function that existed before "dplyr"'s left_join(). Again, for the moment, the functions in "dplyr" are not working with the spatial versions of data frames. Hopefully someone will extend this functionality soon.

For the moment, then we use merge with a similar syntax to left_join(). This gives us a SpatialPolygonsDataFrame with a new column, "n" the number of crimes in the neighborhood.

In [ ]:
head(totals)

In [ ]:
map_totals = merge(nbhds,totals,by="NTACode")
class(map_totals)

In [ ]:
head(map_totals)

In [ ]:
head(as.data.frame(map_totals))